In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/long-practice/mlflow-basecode.git

Cloning into 'mlflow-basecode'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 55 (delta 17), reused 40 (delta 10), pack-reused 0
Receiving objects: 100% (55/55), 115.10 KiB | 1.60 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [3]:
%cd mlflow-basecode

/content/mlflow-basecode


In [4]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [5]:
!pip install optuna --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.9 MB/s eta 0:00:00


In [6]:
from src.pipeline import Pipeline
from preprocessing.preprocess import Preprocess

In [7]:
pipe = Pipeline(train, test)
pipe.run()

[I 2023-10-02 05:35:50,089] A new study created in memory with name: no-name-402ced5c-b7be-455f-a8df-09590fc609b2


Preprocessing Set Missing Value
Preprocessing Set Missing Value
Preprocessing Remove Outlier
column name is : id
lower_bound is : -2703.0
upper_bound is : 8109.0
----------------------------------------
column name is : CementComponent
lower_bound is : -28.25000000000003
upper_bound is : 616.95
----------------------------------------
column name is : BlastFurnaceSlag
lower_bound is : -183.89999999999998
upper_bound is : 306.5
----------------------------------------
column name is : FlyAshComponent
lower_bound is : -118.5
upper_bound is : 197.5
----------------------------------------
column name is : WaterComponent
lower_bound is : 149.75
upper_bound is : 217.35000000000002
----------------------------------------
column name is : SuperplasticizerComponent
lower_bound is : -12.075000000000001
upper_bound is : 20.125
----------------------------------------
column name is : CoarseAggregateComponent
lower_bound is : 775.0000000000001
upper_bound is : 1210.1999999999998
----------------

[I 2023-10-02 05:35:51,575] Trial 0 finished with value: 13.5198244303766 and parameters: {'max_depth': 5, 'learning_rate': 0.35375047395712206, 'n_estimators': 503, 'min_child_weight': 8, 'gamma': 0.5551835528804246, 'subsample': 0.8597497693464806, 'colsample_bytree': 0.4378249330206244, 'reg_alpha': 0.14646434410492595, 'reg_lambda': 0.5001541179436843}. Best is trial 0 with value: 13.5198244303766.
[I 2023-10-02 05:35:53,645] Trial 1 finished with value: 32.01613031664293 and parameters: {'max_depth': 5, 'learning_rate': 0.5983257793734648, 'n_estimators': 700, 'min_child_weight': 3, 'gamma': 0.3124510933221293, 'subsample': 0.23108416052786543, 'colsample_bytree': 0.8491726133844923, 'reg_alpha': 0.4270254570010241, 'reg_lambda': 0.39968307336181025}. Best is trial 0 with value: 13.5198244303766.
[I 2023-10-02 05:36:05,542] Trial 2 finished with value: 16.62063257347899 and parameters: {'max_depth': 13, 'learning_rate': 0.666370595813044, 'n_estimators': 687, 'min_child_weight': 9

In [8]:
import pickle

xgb_model = None
with open('./artifact/XGB_model.pkl', 'rb') as f:
    xgb_model = pickle.load(f)
submission = pd.read_csv('./data/sample_submission.csv')
submission['Strength'] = xgb_model.predict(test)

In [9]:
submission

,id,Strength
0,5407,60.098377
1,5408,20.706198
2,5409,29.460512
3,5410,47.716866
4,5411,3.987035
...,...,...
3600,9007,27.421167
3601,9008,35.043701
3602,9009,50.556957
3603,9010,42.603149


In [10]:
submission.to_csv('./prediction/submission.csv', index=False)